In [3]:
# [神操作：教你用Python识别恶意软件](https://mp.weixin.qq.com/s/kw5cOTIqisRuKYz9UNCbrQ)
import pefile

pe = pefile.PE('./ircbot.exe')

In [4]:
# 从PE文件五个不同的节中提取了数据：.text、.rdata、.data、.idata和.reloc。输出是以五元组的形式给出，每提取一个PE节对应一个元素。每一行的第一个条目标识PE节。（你可以忽略一系列的\x00空字节，它们只是C语言样式的空字符串终止符。）其余字段告诉我们，一旦将每个节被加载到内存中，它的内存利用率将是多少，以及一旦被加载，它将在内存中的何处被找到。
for section in pe.sections:
    print(section.Name, hex(section.VirtualAddress), hex(section.Misc_VirtualSize), section.SizeOfRawData)

b'.text\x00\x00\x00' 0x1000 0x32830 207360
b'.rdata\x00\x00' 0x34000 0x427a 17408
b'.data\x00\x00\x00' 0x39000 0x5cff8 10752
b'.idata\x00\x00' 0x96000 0xbb0 3072
b'.reloc\x00\x00' 0x97000 0x211d 8704


#### 例如，0x1000是加载这些节的虚拟内存地址基址，也可以将其视为节的内存地址基址。在虚拟大小（virtual size）字段中的0x32830指定了节被加载后所需的内存大小。第三个字段中的207360表示该节将在该内存块中所占用的数据量。

In [2]:
# 从ircbot.exe中提取导入信息
# 列出二进制文件将加载的DLL文件，以及它将在这些DLL文件中所请求的函数调用
import pefile
pe = pefile.PE("ircbot.exe")
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print(entry.dll)
    for function in entry.imports:
        print('\t', function.name)

b'KERNEL32.DLL'
	 b'GetLocalTime'
	 b'ExitThread'
	 b'CloseHandle'
	 b'WriteFile'
	 b'CreateFileA'
	 b'ExitProcess'
	 b'CreateProcessA'
	 b'GetTickCount'
	 b'GetModuleFileNameA'
	 b'GetSystemDirectoryA'
	 b'Sleep'
	 b'GetTimeFormatA'
	 b'GetDateFormatA'
	 b'GetLastError'
	 b'CreateThread'
	 b'GetFileSize'
	 b'GetFileAttributesA'
	 b'FindClose'
	 b'FileTimeToSystemTime'
	 b'FileTimeToLocalFileTime'
	 b'FindNextFileA'
	 b'FindFirstFileA'
	 b'ReadFile'
	 b'SetFilePointer'
	 b'WriteConsoleA'
	 b'GetStdHandle'
	 b'LoadLibraryA'
	 b'GetProcAddress'
	 b'GetModuleHandleA'
	 b'FormatMessageA'
	 b'GlobalUnlock'
	 b'GlobalLock'
	 b'UnmapViewOfFile'
	 b'MapViewOfFile'
	 b'CreateFileMappingA'
	 b'SetFileTime'
	 b'GetFileTime'
	 b'ExpandEnvironmentStringsA'
	 b'SetFileAttributesA'
	 b'GetTempPathA'
	 b'GetCurrentProcess'
	 b'TerminateProcess'
	 b'OpenProcess'
	 b'GetComputerNameA'
	 b'GetLocaleInfoA'
	 b'GetVersionExA'
	 b'TerminateThread'
	 b'FlushFileBuffers'
	 b'SetStdHandle'
	 b'IsBadWritePtr'
	

#### 输出的前几行告诉我们，恶意软件将使用WriteFile写入文件，使用CreateFileA打开文件，并使用CreateProcessA创建新的进程。虽然这些只是关于恶意软件的基本信息，但它们是了解恶意软件更为详细行为的开始。

In [3]:
#节表
for section in pe.sections:
    print(section)
print(hex(pe.sections[0].Characteristics))

[IMAGE_SECTION_HEADER]
0x1D8      0x0   Name:                          .text
0x1E0      0x8   Misc:                          0x32830   
0x1E0      0x8   Misc_PhysicalAddress:          0x32830   
0x1E0      0x8   Misc_VirtualSize:              0x32830   
0x1E4      0xC   VirtualAddress:                0x1000    
0x1E8      0x10  SizeOfRawData:                 0x32A00   
0x1EC      0x14  PointerToRawData:              0x400     
0x1F0      0x18  PointerToRelocations:          0x0       
0x1F4      0x1C  PointerToLinenumbers:          0x0       
0x1F8      0x20  NumberOfRelocations:           0x0       
0x1FA      0x22  NumberOfLinenumbers:           0x0       
0x1FC      0x24  Characteristics:               0x60000020
[IMAGE_SECTION_HEADER]
0x200      0x0   Name:                          .rdata
0x208      0x8   Misc:                          0x427A    
0x208      0x8   Misc_PhysicalAddress:          0x427A    
0x208      0x8   Misc_VirtualSize:              0x427A    
0x20C      0xC   Vi

In [4]:

#导入表
for importeddll in pe.DIRECTORY_ENTRY_IMPORT:
    print(importeddll.dll, ':')
    for importedapi in importeddll.imports:
        print('\t', hex(importedapi.address), importedapi.name)

b'KERNEL32.DLL' :
	 0x496230 b'GetLocalTime'
	 0x496234 b'ExitThread'
	 0x496238 b'CloseHandle'
	 0x49623c b'WriteFile'
	 0x496240 b'CreateFileA'
	 0x496244 b'ExitProcess'
	 0x496248 b'CreateProcessA'
	 0x49624c b'GetTickCount'
	 0x496250 b'GetModuleFileNameA'
	 0x496254 b'GetSystemDirectoryA'
	 0x496258 b'Sleep'
	 0x49625c b'GetTimeFormatA'
	 0x496260 b'GetDateFormatA'
	 0x496264 b'GetLastError'
	 0x496268 b'CreateThread'
	 0x49626c b'GetFileSize'
	 0x496270 b'GetFileAttributesA'
	 0x496274 b'FindClose'
	 0x496278 b'FileTimeToSystemTime'
	 0x49627c b'FileTimeToLocalFileTime'
	 0x496280 b'FindNextFileA'
	 0x496284 b'FindFirstFileA'
	 0x496288 b'ReadFile'
	 0x49628c b'SetFilePointer'
	 0x496290 b'WriteConsoleA'
	 0x496294 b'GetStdHandle'
	 0x496298 b'LoadLibraryA'
	 0x49629c b'GetProcAddress'
	 0x4962a0 b'GetModuleHandleA'
	 0x4962a4 b'FormatMessageA'
	 0x4962a8 b'GlobalUnlock'
	 0x4962ac b'GlobalLock'
	 0x4962b0 b'UnmapViewOfFile'
	 0x4962b4 b'MapViewOfFile'
	 0x4962b8 b'CreateFileMapp

In [8]:
#从入口点开始反汇编
ep = pe.OPTIONAL_HEADER.AddressOfEntryPoint
print(hex(ep))
ep_ava = ep + pe.OPTIONAL_HEADER.ImageBase
data = pe.get_memory_mapped_image()[ep:ep+20]
offset = 0
print('len(data) = {}'.format(len(data)))
while offset < len(data):
  i = pydasm.get_instruction(data[offset:], pydasm.MODE_32)
  #打印不换行，加,
  print(hex(ep + offset), '\t',)
  print(pydasm.get_instruction_string(i, pydasm.FORMAT_INTEL, ep_ava+offset))
  offset += i.length

0xcc00ffee
len(data) = 0


In [10]:
#导出符号
#例如calc.exe无导出信息则不存在DIRECTORY_ENTRY_EXPORT变量会出错
for exp in pe.DIRECTORY_ENTRY_EXPORT.symbols:
    print(hex(pe.OPTIONAL_HEADER.ImageBase + exp.address), exp.name, exp.ordinal)

AttributeError: 'PE' object has no attribute 'DIRECTORY_ENTRY_EXPORT'

In [12]:
#改写PE信息
pe.OPTIONAL_HEADER.AddressOfEntryPoint = 0xdeadbeef
pe.write(filename='file_to_write.exe')